In [2]:
from bs4 import BeautifulSoup
import requests
import json
import re

#import gensim
#import nltk
#from nltk import tokenize
#nltk.__version__

In [28]:
"""
Function to get links

"""

#URLs
main_url= 'http://www.sci-news.com'
paleo_url= '/news/paleontology/page/'

#Get links from sections
def getLinksfromPage(url):
    r= requests.get(url)
    soup= BeautifulSoup(r.text, 'html.parser')
    
    #Get grid with title and link
    grids= soup.body.find('div', {'id': 'wrapper'}).section.div.div.div.find('div', {'class': 'single-wrapper'}).section.div
    top= grids.find('div', {'class': 'related-post-wrapper-block'}).div.find('div', {'class': 'recentpost'}).find('div', {'class': 'content-heading'}).h2.a['href']
    bottom= grids.find('div', {'class': 'bottom-recentpost-wrapper-cat'}).find_all('div', class_='bottom-archive')
    #print(top)
    #print(bottom)
    
    #Links
    links= []
    links.append(top)
    for bot in bottom:
        links.append(
            bot.find('div', {'class': 'post-content-holder'}).find('div', {'class': 'bottom-content-heading-0'}).h2.a['href']
        )
    

    return links

getLinksfromPage('http://www.sci-news.com/news/paleontology/page/53')

['http://www.sci-news.com/paleontology/earths-largest-mass-extinction-dated.html',
 'http://www.sci-news.com/paleontology/fossils-reveal-oldest-known-hairy-microbes.html',
 'http://www.sci-news.com/paleontology/gigantopithecus-diet-revealed.html',
 'http://www.sci-news.com/paleontology/giant-kraken-discovered.html',
 'http://www.sci-news.com/paleontology/new-family-of-wasps-discovered-in-amber.html',
 'http://www.sci-news.com/paleontology/new-devonian-predatory-fish-discovered.html',
 'http://www.sci-news.com/paleontology/early-cretaceous-birds-with-crops-found.html',
 'http://www.sci-news.com/paleontology/tibet-a-homeland-for-ice-age-giants.html',
 'http://www.sci-news.com/paleontology/new-milestone-in-mammal-evolution.html',
 'http://www.sci-news.com/othersciences/anthropology/1-9-million-years-of-cooking.html',
 'http://www.sci-news.com/paleontology/oldest-fossils-discovered.html',
 'http://www.sci-news.com/othersciences/anthropology/oldest-evidence-of-nails-in-modern-primates.html'

In [27]:
"""
Function to get article content

"""

r2= 'http://www.sci-news.com/paleontology/aquilolamna-milarcae-09470.html'

def getArticlefromLink(art_link):
    r= requests.get(art_link)
    soup= BeautifulSoup(r.text, 'html.parser')
    #Main
    main= soup.body.find('div', {'id': 'wrapper'}).section.div.div.div.find('div', {'id': 'content'})
    #print(main)
    
    #try:
    #Header
    title= main.header.h1.text
    #try:
    author= main.header.div.find('div', class_='metainfoleft').div.a.text
    #except:
        #author= main.header.div.find('div', class_='metainfoleft').div.span.text
    date= main.header.div.find('div', class_='metainfoleft').div.span.text.split('by')[0].strip()
    #Body
    body= main.section.find_all('p')
    body2= ''
    for par in body[:-1]:
        if(not 'Image credit:' in par.text):
            body2= body2+par.text+' '
    body2= body2.strip()
    
    if(not 'doi' in body[-1].text):
        body2= body2+body[-1].text
        body2= body2.strip()
        
    """except:
        #Header
        try:
            title= main.div.find('div', class_='main-content').h2.text
        except:
            return None
        try:
            author= main.div.find('div', class_='post-info').find('span', class_='byline').a.text
        except:
            author= main.div.find('div', class_='post-info').find('span', class_='byline').span.text
        date= main.div.find('div', class_='post-info').find('span', class_='date').text
        #Body
        body= main.find('div', class_='main-content').find_all('p')
        body2= ''
        for par in body[:-1]:
            body2= body2+par.text+' '    
        body2= body2.strip()"""
    
    #Remove signs
    body2= re.sub(r'[\/#!$%\^&\*;:{}=\[\]_`~()“”…—\"\"‘’\'\'>–]', '', body2)
    body2= re.sub(r'[-]', ' ', body2)
    
    doc= {
        'title': title,
        'author': author,
        'date': date,
        'doc': body2
         }

    return(doc)

getArticlefromLink(r2)

{'title': ' Cretaceous Plankton-Eating Shark Had Long, Wing-Like Fins',
 'author': 'Enrico de Lazaro',
 'date': 'Mar 19, 2021',
 'doc': 'A new species of shark with hypertrophied, slender pectoral fins has been identified from the fossilized remains discovered in northern Mexico. The newly identified shark species, named Aquilolamna milarcae, swam in the Late Cretaceous oceans, approximately 93 million years ago. The complete specimen was found in 2012 in Vallecillo, Mexico, a locality yielding remarkably preserved fossils, said lead author Dr. Romain Vullo from the University of Rennes and the CNRS and colleagues. This site, already famous for its many fossils of ammonites, bony fish and other marine reptiles, is most useful for documenting the evolution of oceanic animals. Aquilolamna milarcae belongs to Lamniformes, an order of sharks in the subclass Elasmobranchii. Elasmobranchs are the highly successful group of cartilaginous fishes, including sharks, skates and rays, the paleonto

In [31]:
"""
Get links and articles content

"""

all_arts= []
for page_i in range(1,54):
    #Join main url and page number
    page_url= main_url+paleo_url+str(page_i)
    #Get links from page i
    all_links= getLinksfromPage(page_url)
    
    #Get articles from page i
    for link_i in all_links:
        all_arts.append(getArticlefromLink(link_i))

#423 total articles
print(len(all_arts))

1053


In [30]:
print(all_arts)

[{'title': ' Meet Yamatosaurus izanagii, New Hadrosaur from Japan', 'author': 'Enrico de Lazaro', 'date': 'Apr 28, 2021', 'doc': 'Yamatosaurus izanagii roamed our planet 71.8 million years ago during the Late Cretaceous epoch. Hadrosaurs, or duck billed dinosaurs, were successful herbivorous dinosaurs during the Cretaceous period. Their fossil record is known mainly from the Early Cretaceous deposits of Europe and eastern Asia, whereas derived forms are known from the Late Cretaceous rocks of all continents except Australia and the Indian subcontinent. Uniquely adapted to chewing, hadrosaurs had hundreds of closely spaced teeth in their cheeks. As their teeth wore down and fell out, new teeth in the dental battery, or rows of teeth below existing teeth, grew in as replacements. The dental structure of Yamatosaurus izanagii distinguishes it from known hadrosaurs, said Dr. Anthony Fiorillo, a paleontologist in the Institute for the Study of Earth and Man at Southern Methodist University 

In [32]:
"""
Save json

"""

#all_arts= [art for art in all_arts if art]
#397 articles
print(len(all_arts))

#Save json
paleo= {'paleontology': all_arts}
with open('paleo2.json', 'w') as fp:
    json.dump(paleo, fp)

1053


In [2]:
"""
Open json

"""

#Open json
with open('paleo.json') as input_file:
    data = json.load(input_file)

data= data['paleontology']
print(len(data))

397


In [3]:
"""
Get content

"""

arts_content= []
for dat in data:
    #Clean text
    #sentences= tokenize.sent_tokenize(dat['doc'])
    #for sent in sentences:
    #cl= re.sub(r'[\.,]', '', sent).lower()
    cl= re.sub(r'[\.,]', '', dat['doc']).lower()
    #Nltk tokenize
    tokens= tokenize.word_tokenize(cl)
    textf= nltk.Text(tokens)
    arts_content.append(textf)
    
print(len(arts_content))

397


In [10]:
"""
Nltk regex: Cambrian explosion and trilobites
Cambrian explosion:
- Paleontologists are still discussing whether the evolutionary event was a real 
sudden change or if it simply looks that way because of incomplete fossil sampling.
- Unlike the single celled animals and microbes that lived before the geological age, 
these creatures had it all heads tails abdomens shells and lots of tiny little legs.

Trilobites or complex fossils:
- Few mentions.

"""

for art in arts_content:
    art.findall(r"(<\w+>{20} <organisms?> <\w+>{,20} <complex|complexity> <\w+>{60})|<\w+>{20} <complex|complexity> <\w+>{,20} <organisms?> <\w+>{60}")
    #print() complex



























































































































































































































































































among the earliest organisms to thrive on land brooks hays writes for
united press international during the period when this organism
existed life was almost entirely restricted to the oceans nothing more
complex than simple mossy and lichen like plants had yet evolved on
the land smith said in a statement but before there could be flowering
plants or trees or the animals that depend on them the processes of
rot and soil formation needed to be established by examining hundreds
of the tiny fossils smith found that tortotubus may have helped set




















































































































In [12]:
"""

"""

for art in arts_content:
    art.findall(r"(<\w+>{20} <carbon> <\w+>{60})")
    #print() complex



by chance the team combined the standard physical study of the fossils
with chemical analyses using rare earth elements stable carbon and
oxygen isotopes and charcoal concentrations reports juliet eilperin
for the washington post none of the physical evidence conclusively
suggested that these organisms came to be fossilized together so we
turned to geochemistry to see if that could help us says celina suarez
a paleontologist at the university of arkansas and co author of the
research in the statement the





from an ecosystem with an open canopy for the second method the
researchers checked the ratio of a pair of carbon isotopescarbon 13
and carbon 12to infer how much sun beat down on a leaf when it was
alive if a collection of a forests leaves has roughly consistent
ratios of carbon 12 to carbon 13 isotopes then the forest probably had
an open canopy but if the forests leaves display a big range of carbon
isotope ratios that suggests a; the fossilized angiosperms that
repopulated s

scientists were able to assign an approximate age of
















human artifacts and about 66 pounds of bones 70 percent of which came
from the yellow footed rock wallaby using carbon dating giles built up
a timeline of how often humans used the rock shelter over the course
of 50000 years the research showed that usage spiked around 40000
years ago then diminished 35000 years ago when the region became very
arid people returned about 17000 years ago what is different about it
is its the southern most oldest site in







but even then he wasnt entirely convinced about the cause of these
extinctions the leading hypothesis holds massive eruptions blasted
carbon dioxide into earths atmosphere cranking up global temperatures
within a relatively short period of time such a sudden change the
theory goes would have killed off terrestrial species like the huge
ancestors of crocodiles and large tropical amphibians and opened the
door for dinosaurs to evolve olsen who discovered his first d

In [21]:
"""
TF Visualizer

"""

aer= []
for i in art_content:
    corpus= list(gensim.utils.tokenize(i))
    aer.append(corpus)
print(aer)
#model = gensim.models.Word2Vec(iter = 5, size = 32)
model= gensim.models.Word2Vec(aer, min_count= 5, size= 32)
vocab= list(model.wv.vocab)
print(len(vocab))
# save memory
vectors = model.wv
#del model
vectors.save_word2vec_format("vect.txt", binary = False)

[['when', 'paleontologists', 'from', 'the', 'new', 'mexico', 'museum', 'of', 'natural', 'history', 'and', 'science', 'uncovered', 'the', 'fossil', 'of', 'a', 'foot', 'long', 'shark', 'in', 'they', 'began', 'calling', 'it', 'godzilla'], ['the', 'nickname', 'fit', 'the', 'prehistoric', 'beasts', 'unusual', 'features', 'including', 'rows', 'of', 'short', 'squat', 'teeth', 'and', 'two', 'long', 'reptilian', 'spines', 'on', 'its', 'back'], ['now', 'after', 'seven', 'years', 'of', 'research', 'to', 'catalogue', 'its', 'various', 'characteristics', 'the', 'scientists', 'have', 'determined', 'that', 'the', 'fossil', 'represents', 'a', 'new', 'species'], ['its', 'official', 'name', 'is', 'dracopristis', 'hoffmanorum', 'the', 'museum', 'announced', 'in', 'a', 'statement', 'last', 'week'], ['the', 'first', 'part', 'of', 'its', 'name', 'means', 'dragon', 'shark', 'in', 'latin', 'and', 'the', 'latter', 'hoffmanorum', 'is', 'a', 'tribute', 'to', 'the', 'hoffman', 'family', 'that', 'owns', 'the', 'qu

In [24]:
model.wv.most_similar('fossils')

[('it', 0.3088524043560028),
 ('its', 0.29567354917526245),
 ('fossil', 0.2554177939891815),
 ('walking', 0.22427135705947876),
 ('from', 0.18144172430038452),
 ('hodnett', 0.17023035883903503),
 ('also', 0.1570967733860016),
 ('per', 0.15670239925384521),
 ('dinosaurs', 0.13096080720424652),
 ('paleontologists', 0.12802574038505554)]

'3.5'